In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.weight'] = 'bold'
root_path = "../output/instance"
split_path = "../data/INS"
metric_list = os.listdir(root_path)
rawdata = pd.read_csv("../data/Process/Gene_Entity_v2.csv")
rawdata

,~id,id,blattner_id_secondary:String,EcoGene_Accession:String,name:String,Len:Int,Ori:String,LeftEnd:Int,RightEnd:Int,Cs:Float,...,TCDB_Fam:String,TF_Binding_Sites:String,TIGR_CMR:String,Uber_Operon:String,is_ace:String,GO:String[],nucleic_acid_seq:String,amino_acid_seq:String,label,~label
0,b0001,b0001,NaN,EG11277,thrL,21,Clockwise,190,255,0.00,...,NaN,NaN,NaN,NaN,0,{'GO:0009088': {'go_term_name': 'threonine bio...,ATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCA...,MKRISTTITTTITITTGNGAG,Gene,Gene
1,b0002,b0002,NaN,EG10998,thrA,820,Clockwise,337,2799,0.01,...,NaN,thrA.html,NT01EC0001,2530.0,1,{'GO:0004072': {'go_term_name': 'aspartate kin...,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,Gene,Gene
2,b0003,b0003,NaN,EG10999,thrB,310,Clockwise,2801,3733,0.06,...,NaN,NaN,NT01EC0002,2530.0,1,"{'GO:0005737': {'go_term_name': 'cytoplasm', '...",ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...,Gene,Gene
3,b0004,b0004,NaN,EG11000,thrC,428,Clockwise,3734,5020,0.08,...,NaN,NaN,NT01EC0003,2530.0,1,{'GO:0004795': {'go_term_name': 'threonine syn...,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...,Gene,Gene
4,b0005,b0005,NaN,EG14384,yaaX,98,Clockwise,5234,5530,0.11,...,NaN,b0005.html,NaN,NaN,0,none,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYW...,Gene,Gene
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4497,b4710,b4710,NaN,EG60057,insB9,167,Counterclockwise,257923,258426,5.56,...,NaN,NaN,NaN,NaN,0,none,ATGCCGGGCAACAGCCCGCATTATGGGCGTTGGCCTCAACACGATT...,MPGNSPHYGRWPQHDFTSLKKLRPQSVTSRIQPGSDVIVCAEMDEQ...,Gene,Gene
4498,b4711,b4711,NaN,EG60058,insH21,326,Clockwise,1299567,1300547,28.00,...,NaN,NaN,NaN,NaN,0,none,ATGAGTCATCAACTTACCTTCGCCGACAGTGAATTCAGCAGTAAGC...,MSHQLTFADSEFSSKRRQTRKEIFLSRMEQILPWQNMVEVIEPFYP...,Gene,Gene
4499,b4712,b4712,NaN,EG31189,agrA,84,Clockwise,3648063,3648146,78.59,...,NaN,NaN,NaN,NaN,0,none,TTAGATGATGGCTATCTCACTCCAGTCAGAGCCACCAACTCAGGGC...,none,Gene;sRNA,Gene;sRNA
4500,b4713,b4713,NaN,EG31190,agrB,84,Clockwise,3648294,3648377,78.60,...,NaN,NaN,NaN,NaN,0,none,TTAGATAAGAACTCTCTCACTCCAGCCAGAGCCACCAACTCAGGGC...,none,Gene;sRNA,Gene;sRNA


In [2]:
select_idx = []
for i in range(1, 148):  # 假设您要迭代从1到100的整数
    idx = metric_list[i-1].replace(".npz", "")  # 因为列表索引是从0开始的
    loaded_data = np.load(os.path.join(root_path, metric_list[i-1]))
    labels = loaded_data['label']
    predict = np.argmax(loaded_data['prob'], axis=1)
    
    # 计算F1分数
    f1 = f1_score(labels, predict, average='macro')
    select_idx.append(i) if f1>0.6 else None
len(select_idx)
select_idx = select_idx[:100]

In [3]:
result = []
for i in select_idx:  # 假设您要迭代从1到100的整数
    idx = metric_list[i-1].replace(".npz", "")  # 因为列表索引是从0开始的
    loaded_data = np.load(os.path.join(root_path, metric_list[i-1]))
    labels = loaded_data['label']
    predict = np.argmax(loaded_data['prob'], axis=1)
    test_dataset = pd.read_csv(os.path.join(split_path, metric_list[i-1].replace(".npz",""),"test_node.csv")).sort_values("nid")
    test_dataset.index = range(len(test_dataset))
    misclassified_indices = np.where((labels == 0) & (predict == 1))[0]
    result.append(test_dataset.iloc[misclassified_indices])

In [4]:
res = pd.concat(result, ignore_index=True)
counts = res['nid'].value_counts()
res['Repeat'] = res['nid'].map(counts)
res['In_Test'] = 0
res = res.drop_duplicates(keep="first")
len(res)

68

In [5]:
for i in select_idx:  # 假设您要迭代从1到100的整数
    idx = metric_list[i-1].replace(".npz", "")  # 因为列表索引是从0开始的
    loaded_data = np.load(os.path.join(root_path, metric_list[i-1]))
    labels = loaded_data['label']
    predict = np.argmax(loaded_data['prob'], axis=1)
    test_dataset = pd.read_csv(os.path.join(split_path, metric_list[i-1].replace(".npz",""),"test_node.csv")).sort_values("nid")
    test_dataset.index = range(len(test_dataset))
    res['In_Test'] = res.apply(lambda x: x['In_Test'] + int(test_dataset[test_dataset['id']==x.id].empty), axis=1)
res.head(5)

,id,~label,label,nid,Repeat,In_Test
0,b2667,0,Gene,709,15,68
1,b0240,0,Gene,1962,8,74
2,b0101,0,Gene,2052,4,70
3,b2068,0,Gene,2635,2,71
5,b4532,0,Gene,1038,14,67


In [6]:
output = pd.merge(rawdata,res[["id","Repeat","In_Test"]],on="id").sort_values("Repeat",ascending=False)
output.to_excel("../output/node_selected.xlsx", index=False)
output

,~id,id,blattner_id_secondary:String,EcoGene_Accession:String,name:String,Len:Int,Ori:String,LeftEnd:Int,RightEnd:Int,Cs:Float,...,TIGR_CMR:String,Uber_Operon:String,is_ace:String,GO:String[],nucleic_acid_seq:String,amino_acid_seq:String,label,~label,Repeat,In_Test
12,b0483,b0483,NaN,EG13245,ybaQ,113,Clockwise,508218,508559,10.95,...,NT01EC0593,NaN,0,{'GO:0043565': {'go_term_name': 'sequence-spec...,ATGAAACAGGCAACAAGAAAACCGACGACACCCGGAGATATTCTTC...,MKQATRKPTTPGDILLYEYLEPLDLKINELAELLHVHRNSVSALIN...,Gene,Gene,24,68
3,b0208,b0208,NaN,EG11649,yafC,304,Counterclockwise,229967,230881,4.95,...,NT01EC0244,NaN,1,"{'GO:0003677': {'go_term_name': 'DNA binding',...",ATGAAAGCCACGTCGGAAGAACTCGCCATTTTTGTTTCGGTCGTCG...,MKATSEELAIFVSVVESGSFSRAAEQLGQANSAVSRAVKKLEMKLG...,Gene,Gene,19,72
58,b3472,b3472,NaN,EG12218,dcrB,185,Clockwise,3609955,3610512,77.77,...,NT01EC4202,2508.0,0,{'GO:0042597': {'go_term_name': 'periplasmic s...,ATGCGCAATCTGGTTAAATATGTCGGAATTGGCCTGCTGGTTATGG...,MRNLVKYVGIGLLVMGLAACDDKDTNATAQGSVAESNATGNPVNLL...,Gene,Gene,18,68
45,b2667,b2667,NaN,EG13524,ygaV,99,Clockwise,2797211,2797510,60.26,...,NT01EC3222,2464.0,0,{'GO:0005622': {'go_term_name': 'intracellular...,ATGACTGAACTCGCGCAATTACAGGCCAGTGCCGAACAGGCAGCGG...,MTELAQLQASAEQAAALLKAMSHPKRLLILCMLSGSPGTSAGELTR...,Gene,Gene,17,68
27,b1477,b1477,NaN,EG13784,yddM,94,Counterclockwise,1552398,1552682,33.44,...,NT01EC1790,NaN,0,{'GO:0043565': {'go_term_name': 'sequence-spec...,ATGAAAATGGCCAATCATCCCCGCCCGGGGGACATTATTCAGGAAT...,MKMANHPRPGDIIQESLDELNVSLREFARAMEIAPSTASRLLTGKA...,Gene,Gene,17,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,b2971,b2971,NaN,EG12991,yghG,136,Counterclockwise,3113067,3113477,67.07,...,NT01EC3589,NaN,0,{'GO:0005886': {'go_term_name': 'plasma membra...,ATGTCGATAAAACAAATGCCAGGGAGGGTATTAATATCGCTATTGT...,MSIKQMPGRVLISLLLSVTGLLSGCASHNENASLLAKKQAQNISQN...,Gene,Gene,1,72
10,b0481,b0481,NaN,EG12454,ybaK,159,Counterclockwise,506603,507082,10.91,...,NT01EC0590,NaN,1,"{'GO:0005737': {'go_term_name': 'cytoplasm', '...",ATGACACCCGCAGTTAAATTACTCGAAAAAAACAAGATTTCGTTTC...,MTPAVKLLEKNKISFQIHTYEHDPAETNFGDEVVKKLGLNPDQVYK...,Gene,Gene,1,69
16,b0947,b0947,NaN,EG13716,ycbX,369,Counterclockwise,1006491,1007600,21.68,...,NT01EC1151,NaN,1,{'GO:0003824': {'go_term_name': 'catalytic act...,GTGGCGACATTAATCCGGCTTTTTATTCATCCTGTTAAATCGATGC...,MATLIRLFIHPVKSMRGIGLTHALADVSGLAFDRIFMITEPDGTFI...,Gene,Gene,1,72
15,b0710,b0710,NaN,EG13305,ybgI,247,Clockwise,742827,743570,16.00,...,NT01EC0876,NaN,1,"{'GO:0005737': {'go_term_name': 'cytoplasm', '...",ATGAAAAACACCGAACTGGAACAACTGATTAACGAAAAACTGAACA...,MKNTELEQLINEKLNSAAISDYAPNGLQVEGKETVQKIVTGVTASQ...,Gene,Gene,1,70
